# Welcome to medicinal chemistry workshop!

### Story 
_You are a chemist working in the laboratory of a large pharmaceutical company. **Your task is to study drug compounds and compare their properties to those of aspirin.** Water solubility is particularly important, as the drug compound needs to dissolve well in blood. The biological effect and metabolism of the drug compound can be modeled later._

_You have already studied many different promising drug compounds without success in the laboratory using demanding solubility tests that depend on the acidity and temperature of the solutions. The project deadline for completing the solubility tests is approaching. Therefore, new ways must be found to determine the best possible drug compounds from the sample for further research and experimentation._

_Fortunately, you have access to various laboratory results and modeling tools that pharmaceutical and chemical software developers have been working on for decades. It's time to dive into the world of cheminformatics and solve the water solubility tests before the deadline!_


## Technical instructions
This work is done in a Jupyter notebook. This notebook is an environment that contains cells that make it **modular**. This modularity means that the computer reads each cell individually, but in top-down order. The cells can be divided into two types in this work instruction:

- Code cells, in which commands can be written. Often, before this, they load various libraries that are used to execute commands and processes.
- Markdown language cells, which contain text that can be formatted like in a text editor, but with certain **commands** using different characters either before or on both sides of a word.

You can edit all cells by double-clicking on the cell and writing commands on the lines or pasting copied text into them with the command. You can run each cell and see the effect of your changes by pressing: **CTRL + ENTER**

If you encounter technical problems (for example, a cell does not work), this may be because you have run the cell in a different order than normal. Try to run the cells from top to bottom. If this is not the cause of the problem, use the save command and then press **CTRL + SHIFT + R**. This will reset the notebook. Alternatively, you can run the 'Restart the kernel' command from the top bar.

---

# PART I

### Recap of how SMILES work

**Table 1.** Examples of SMILES rules
| Symbol | Example | Description |
| --- | --- | --- |
| Chemical symbol | `O` | This is a molecule, water. Elements are expressed as chemical symbols, as in the periodic table. Hydrogens are not indicated but are automatically interpreted as part of the compound based on the typical valence of the element, meaning the number of bonds. See the code to find out how to display them. |
| Covalent bond | `C=C` | Single bonds are not shown, but double and triple bonds are shown with the symbols = and #. |
| Hydrogen | `C` | When determining chirality, C corresponds to methane, that is CH4. |
| Branches | `CCCC(C)CC` | The brackets () indicate the branch, and the symbols within the brackets indicates which branch is involved. |
| Ring structure | `C1CCCCC1` or `C1CCC(C(C1)Cl)Cl` | The numbers 1-9 indicate the position of the atom in the ring structure. Substituents replacing hydrogen atoms can be placed in brackets. NOTE: Cl is a chlorine atom. |
| Aromaticity | `c1ccccc1` | The chirality of the ring structure can be represented by changing the letters to lowercase. |
| Chiraliry | `C[C@@H](C(=O)O)N` | L-alanine, where @@ after the carbon indicates that it is chiral. |
| Isotopes | `[13C]` | An ethane molecule consisting of C13 isotopes. The numbers before the element symbol indicate the isotope. |
| Electric charge and radicals | `[NH4+].[Cl-]` or `C[CH2].[CH3]` | Reservation + or -, and radicals are often read using special algorithms in software that interprets SMILES strings. There is no established notation for them. |

### 1. Drawing compounds using SMILES strings

In [ ]:
# Using the code below, you can draw a compound by entering the SMILES string in the
# smiles = ' ' field and running the cell by pressing CTRL + Enter.
# Start by trying one of the SMILES strings from the table above.

# Import needed modules from libraries
from rdkit import Chem
from rdkit.Chem import rdDepictor
from rdkit.Chem.Draw import IPythonConsole
from rdkit.Chem.Draw import rdMolDraw2D
from IPython.display import SVG

smiles = 'C'  # Move the desired SMILES string between the ' ' characters to replace C. Finally, press CTRL + ENTER to run the cell.
m = Chem.MolFromSmiles(smiles)

# m = Chem.AddHs(m) # You can use this command to visualize all hydrogen atoms, if you wish. Remove the first # character to activate the setting.

# Create a function moltosvg that creates an SVG image of a structural formula from a SMILES string.
def moltosvg(mol, molSize=(350, 300), kekulize=True):
    mc = Chem.Mol(mol.ToBinary())
    if kekulize:
        try:
            Chem.Kekulize(mc)
        except:
            mc = Chem.Mol(mol.ToBinary())
    if not mc.GetNumConformers():
        rdDepictor.Compute2DCoords(mc)
    drawer = rdMolDraw2D.MolDraw2DSVG(molSize[0], molSize[1])
    drawer.DrawMolecule(mc)
    drawer.FinishDrawing()
    svg = drawer.GetDrawingText()
    svg = drawer.GetDrawingText()
    return SVG(svg)
    

structuralFormula = moltosvg(m)  # Define the structural formula variable and set its value to an SVG image using the moltosvg function.
display(structuralFormula)       # Display the structural formula image.

###  Guestion 2.
**Which compound did you draw, and what functional groups does the compound have?**

These questions are answered on the Microsoft Forms form.

---

## 2. Creating a union table
Let's test how union tables work. Select a more complex SMILES and try to create a union table for it using the following code. You can also try this with multiple SMILES. The computer starts indexing from zero, meaning that the first carbon is in the table and the structure is numbered 0 in the image.

In [ ]:
# Import needed modules from libraries
from rdkit import Chem
from rdkit.Chem import rdDepictor
from rdkit.Chem.Draw import IPythonConsole
from rdkit.Chem.Draw import rdMolDraw2D
from IPython.display import SVG
import numpy as np
import pandas as pd

mol = Chem.MolFromSmiles('C') # Repleace C with a more complex SMILES string

# Define settings for the rdkit drawing function.
d2d = rdMolDraw2D.MolDraw2DSVG(300,300)
d2d.drawOptions().addAtomIndices = True  # Add carbon numbering to the image. Numbering starts at 0 on the computer.
d2d.DrawMolecules([mol])
d2d.FinishDrawing()
svg = SVG(d2d.GetDrawingText())

num_atoms = mol.GetNumAtoms()
connectivity_matrix = np.zeros((num_atoms, num_atoms), dtype="f")
pd.options.display.float_format = lambda x: ('%g' % x)


# Create a union table using a for loop.
for bond in mol.GetBonds():
    begin_atom_idx = bond.GetBeginAtomIdx()    
    end_atom_idx = bond.GetEndAtomIdx()
    bond_type = (bond.GetBondTypeAsDouble())    
    connectivity_matrix[begin_atom_idx][end_atom_idx] = bond_type
    connectivity_matrix[end_atom_idx][begin_atom_idx] = bond_type
    
df = pd.DataFrame(connectivity_matrix)

# Print the structural formula and the connection table.
display(svg, df)

### Question 3.
**What is the bond between the second and third carbons?**

----

## 3. Let's learn about stereoisomerism

In [ ]:
# With this code, you can draw stereochemical compounds
# First, mol1 and mol2 are defined as SMILES

mol1 = Chem.MolFromSmiles('C/C=C/C') # Note that this only works for isomeric SMILES strings.
mol2 = Chem.MolFromSmiles('C[C@H](O)[C@@H](N)C(O)=O')


d2d = rdMolDraw2D.MolDraw2DSVG(600,280,300,280)
d2d.drawOptions().addStereoAnnotation=True # The command marks chiral centers on the drawing.
d2d.DrawMolecules([mol1,mol2]) # RDKit can only print two molecules at a time
d2d.FinishDrawing()
rakenneKaavat = SVG(d2d.GetDrawingText())
display(rakenneKaavat)

### Question 4.
**Is the molecule on the left cis or trans? How many chiral centers are there in the compound on the right?**

---

## 4. Introduction to the PUG-REST method and SMILES

Compounds can be easily imported into the software environment by submitting queries to the PubChem server. This is particularly useful when you want to quickly import information about drug compounds without using search engines by listing the desired molecules as part of the code.

This is especially useful when you want to search for SMILES strings in a database.

### Examples of Medicine
| ACTIVE INGREDIENT        | BRAND NAME                                   | DESCRIPTION                                                                                                              |
| ------------------------ | -------------------------------------------- | ------------------------------------------------------------------------------------------------------------------------ |
| **Aspirin**             | **(Aspirin)**                               | The most widely used anti-inflammatory drug. It prevents blood clots, so it's used in the initial treatment of heart attacks. |
| **Ibuprofen**           | **(Burana)**                                | An anti-inflammatory drug.                                                                                               |
| **Paracetamol**         | **(Panadol)**                               | A pain reliever.                                                                                                       |
| **Morphine**            | **(Morphine)**                              | A powerful pain reliever. Used in hospital care.                                                                        |
| **Cortisone**           | **(Prednisone, Hydrocortisone)**            | Cortisone is used for conditions like asthma, allergies, and skin diseases. Administered through inhalation, as tablets, or as ointment. |
| **Penicillin V**        | **(V-Pen)**                                 | An antibiotic for treating bacterial infections.                                                                         |
| **Cetirizine**          | **(Heinix)**                                | An antihistamine for treating allergies.                                                                                 |
| **Insulin**             | **(Lantus)**                                | A hormone for treating diabetes.                                                                                         |
| **Paclitaxel**          | **(Taxol)**                                 | A chemotherapy drug.                                                                                                     |
| **Sertraline**          | **(Zoloft)**                                | An antidepressant, also used for panic disorder and obsessive-compulsive disorder.                                       |


Select three medicinal substances to study. You can use the table above or enter the names of medicinal substances of your choice. Enter the medicinal substances in English! You can search for the name of the active ingredient of the medicine online. Do not use the brand name of the medicine.

In [ ]:
# Import needed modules from libraries
import requests

# Create a function that retrieves data from the PubChem database using the PUG-REST method.
# Do not modify this part of the code!
def retrieve_names(compound_names):
    pugrest = "https://pubchem.ncbi.nlm.nih.gov/rest/pug"
    pugoper = "property/CanonicalSMILES"
    pugout = "txt"
    results = {}
    for compound_name in compound_names:
        pugin = "compound/name/" + compound_name
        url = f"{pugrest}/{pugin}/{pugoper}/{pugout}"
        response = requests.get(url)
        if response.status_code == 200:
            results[compound_name] = response.text.strip()  # Remove extra spaces.
        else:
            results[compound_name] = f"Error: {response.status_code}"
    return results

# Edit the code line below. Add the English names of the drugs you have selected here, enclosed in quotation marks.
compound_names = ["", "", ""] 
# Only edit the code line above.

# Retrieve SMILES strings using the retrive_names function.
canonical_smiles = retrieve_names(compound_names)

# Print out the results
for compound_name, smiles in canonical_smiles.items():
    print(f"Canonical SMILES for {compound_name}: {smiles}")

### Question 5.
**Which drugs did you choose and what are their SMILES strings?**

---

## 5. Piirretään lääkeaineita

In [ ]:
# Import needed modules from libraries
from rdkit import Chem 
from rdkit.Chem.Draw import IPythonConsole 
from rdkit.Chem import rdDepictor 
from rdkit.Chem.Draw import rdMolDraw2D 
from IPython.display import SVG 

smiles = '' # Testaa hakujesi tuloksia lisäämällä SMILES kerrallaan '' merkkien väliin. 
m = Chem.MolFromSmiles(smiles)
# m = Chem.AddHs(m) # Voit tällä komennolla visualisoida kaikki vedyt, jos haluat. Poista ensimmäinen #-merkki aktivoidaksesi asetuksen


# Luodaan funktio moltosvg_with_indices, joka luo SMILES-merkkijonosta rakennekaavan kuvan SVG-muotoon.
def moltosvg_with_indices(mol, molSize = (350,300), kekulize = True):
    mc = Chem.Mol(mol.ToBinary())
    if kekulize:
        try:
            Chem.Kekulize(mc)
        except:
            mc = Chem.Mol(mol.ToBinary())
    if not mc.GetNumConformers():
        rdDepictor.Compute2DCoords(mc)
    drawer = rdMolDraw2D.MolDraw2DSVG(molSize[0],molSize[1])
    drawer = Chem.Draw.rdMolDraw2D.MolDraw2DSVG(molSize[0], molSize[1])
    drawer.drawOptions().addAtomIndices = True
    drawer.DrawMolecule(mc)
    drawer.FinishDrawing()
    svg = drawer.GetDrawingText()
    return SVG(svg)
    

rakenneKaava = moltosvg_with_indices(m)  # Määritetään rakenneKaava muuttuja ja asetetaan sen arvoksi SVG kuva moltosvg_with_indices funktiolla.
display(rakenneKaava)       # Tulostetaan rakennekaavan kuva.

### Kysymys 6.
**Mitä funktionaalisia ryhmiä löydät lääkeaineista?**

---

## 6. Tutustutaan IUPAC-nimiin ja PubChem tietokantaan

In [ ]:
# Tällaisella koodilla voit hakea erilaisilla triviaali- tai IUPAC-nimillä tiettyjä tietoja PubChem-tietokannasta.

# Import needed modules from libraries
import pubchempy as pcp 
import pandas as pd
pd.options.display.max_colwidth = 1000 # Määritetään pandas-kirjaston sarakkeille oletusarvoa pidempi maksimi pituus.

# Lista esimerkki aineista. Voit lisätä tähän haluamasi yhdisteet.
names = ["water", 
         "benzene", 
         "methanol", 
         "ethene", 
         "ethanol", 
         "propene", 
         "1-propanol", 
         "2-propanol", 
         "butadiene", 
         "1-butanol", 
         "2-butanol", 
         "tert-butanol"]

# Käytetään for-silmukkaa listan läpikäymiseen ja tallennenetaan aineiden ID:t listaan.
# PubChem-tietokannan yhdisteet on indeksoitu (ID) ja CID on lyhenne kemialliselle indeksille.
# Ne ovat uniikkeja nimikkeille ja toimivat selkeämpänä perustana haulle SMILES:n tai muiden tietojen sijaan.
compound_ids = []
for name in names:
    try:
        cp = pcp.get_compounds(name, namespace='name')
        if cp:
            compound_ids.append(cp[0].cid)
    except Exception as e:
        print(f"Error retrieving CID for '{name}', : {e}")
results = pcp.get_properties(['IUPACName', 'MolecularFormula', 'MolecularWeight', 'CanonicalSMILES'], compound_ids)

# Luodaan tuloksista pandas-kirjaston taulukko ja tulostetaan se.
df = pd.DataFrame(results)
display(df)

### Kysymys 7.
**Miten valitsemasi lääkeaineen nimi eroaa IUPAC-nimestä?**

---

# OSA II

QSAR-algoritmiä voidaan hyödyntää lääkeaineiden biologisen aktiivisuuden ennustamiseen. Ihan ensimmäiseksi algoritmia hyödynnettiin silti kiehumispisteen ennustamiseen molekyylirakenteen perusteella. Siksi aloitetaan tässä työpajassa visualisoimalla moolimassan ja kiehumispisteen korrelaatiota. 

### 7. Tutkitaan mooolimassan ja kiehumispisteen korrelaatiota

In [ ]:
# Import needed modules from libraries
import pandas as pd

pd.options.display.max_colwidth = 1000  # Määritetään pandas kirjaston sarakkeille oletusarvoa pidempi maksimipituus.
df = pd.read_csv('../../data/BP.csv')              # Luetaan tiedosto taulukoksi pandas-kirjaston avulla.
display(df)                             # tulostaa taulukon, koska määritelty df tarkoittaa dataframe -komentoa.

In [ ]:
# Import needed modules from libraries
import matplotlib.pyplot as plt

plt.scatter(df.Moolimassa, df.Kiehumispiste_K)      # Tekee verrannon kiehumispisteen (K) suhteesta moolimassaan
plt.xlabel('Moolimassa')                            # Asetetaa x-akselille nimi
plt.ylabel('Kiehumispiste')                         # Asetetaa y-akselille nimi
plt.show()                                          # Tulostetaan kuvaaja

### Kysymys 8.
**Mitä voit päätellä molekyylien kiehumispisteistä ja niiden suhteesta moolimassaan? Miksi sama moolimassa voi johtaa eri kiehumispisteeseen?**

---

### 8. Lineaarisen regressiomallin sovittaminen

Lineeariset sovitukset ovat tuttuja varmasti matematiikan tunneilta. Tämä matemaattinen mallintaminen on yksi keskeisimpiä menetelmiä keminformatiikassa, koska sillä voidaan tarkastella erilaisten muuttujien suhdetta toisiinsa (korrelaatio) alkaen kahdesta tai usemmasta tekijästä. Alla on koodia, jolla voidaan sovittaa regressiomalli datasettiin. Mitä huomaat, kun koodi tulostaa graafin? 

In [ ]:
# Import needed modules from libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression

# Tuodaan dataa .csv tiedostosta taulukoksi
df = pd.read_csv('../../data/BP.csv')

# Sovitetaan lieneaarinen regressiomalli
model = LinearRegression()
X = df[['Moolimassa']]          # Riippumaton muuttuja
y = df[['Kiehumispiste_K']]     # Riippuva muuttuja
model.fit(X, y)

# Piirrä  pistekuvio datasta 
plt.figure()
sns.scatterplot(x='Moolimassa', y='Kiehumispiste_K', data=df, color='tab:blue', label='Datapisteet')

# Piirrä regressiosuora
plt.plot(X, model.predict(X), color='tab:red', label='Lineaarinen regressio')

# Aseta otsikot ja mitat
plt.xlabel('Moolimassa')
plt.ylabel('Kiehumispiste (K)')
plt.title('Lineaarinen regressiomalli')

# Näytä selite
plt.legend()

# Näytä kuvaaja
plt.show()

### Kysymys 9.
**Seuraako punainen regressiosuora datapisteitä tarkasti?**

---

### 9. Lääkeaineiden toimivuuden ennustaminen
#### Tehtäväsi

**Löytää yhdisteitä, joilla on aspiriiniin verrattavissa oleva liukoisuus LogS-arvo -2.1 tutkimalla niitä alla sijaitsevan koneoppimisen mallin avulla.** LogS-arvo kuvaa yhdisteen vesiliukoisuutta ja alla olevassa taulukossa on kerrottu siitä tarkemmin. 

| Liukoisuus (LogS) | LogS < -4            | -4 < LogS < -2            | -2 < LogS < 0 |
| ----------------- | ----------------- | -------------------- | ------------------------- |
| Tulkinta                           | Ei liukene veteen                  | Liukenee melko hyvin                     | Liuekenee erinomaisesti                            |
| Pitoisuus veressä                  | Minimaalinen                       | Hyvä                                     | Korkea, pitoisuuden nousu                          |


On aika tutkia vähintään 5 yhdistettä aloittamalla niiden SMILES-nimistä. Käytä aina valitsemiesi yhdisteiden lisäksi **aspiriiniä** referenssinä:

- Voit etsiä tietokantoja joko vapaasti tai PUG-REST -komennoilla
- Hyödynnä kokeellista dataa yhdisteistä (taulukko).

Arvioi niiden soveltuvuutta lääkeyhdisteiksi erityisesti **liukoisuuden arvon ja Lipinskin viiden säännön suhteen**.

**Tämän työn tarkoitus on kartoittaa pieni otos samaa liukoisuutta omaavista yhdisteistä**, joita esimerkiksi sinä tai muut voivat **hyödyntää lähtökohtana seuraaville keminformatiikan liukoisuusmalleille**. Kaikki lähtee tunnetuista ja varmennetuista datapisteistä! Onnea matkaan!

In [ ]:
# Voit esimerkiksi tutkia vastaako mallin arvot kokeellisten tutkimusten tuloksia. 
# Datasetissä on epäonnistuneiden ennusteiden ja oikeiden ennusteiden tuloksia.

# Tuodaan tarvittavat moduulit kirjastoista.
import numpy as np
import pandas as pd

# Määriteään pandaskirjaston tulostusasetuksia
pd.options.display.max_colwidth = 1000  # Määritetään pandas kirjaston sarakkeille oletusarvoa pidempi maksimipituus.
pd.options.display.max_rows = 500       # Määritetään pandas kirjastolle oletusarvoa enemmän rivejä.

df = pd.read_csv('../../data/kokeellisetLogS_referenssipisteet.csv') # Luetaan tiedosto taulukoksi pandas -ohjelmiston avulla.
df = df.loc[:100, ['Chemical name', 'LogS exp (mol/L)', 'Test', 'SMILES']] # Valitaan tulukosta haluamamme sarakkeet

# Tulostetaan taulukko
display(df)

### 10. Koneoppimisen malli

In [ ]:
# Tuodaan koneoppimisen malli lääkeyhdisteiden vesiliukoisuuden ennustamiseksi. 

# Import needed modules from libraries
from pycaret.classification import *

# Ladataan ennanltaopetettu koneoppimisen malli.
model = load_model("../../data/WaterSoulubility_02_06_2025_model")

Käytä tätä koneoppimisen mallia ennustamaan yhdisteiden liukoisuutta ja soveltuvuutta lääkeaineiksi! Syötä SMILES-jonoja yhdisteille mitä haluat tutkia, mutta muista erottaa ne pilkulla ja rivinvaihdolla.

In [ ]:
# Import needed modules from libraries
from chem_util import descriptors_from_smiles 

# Syötä listaan SMILES-merkkijonoja käyttäen lainausmerkkejä jokaisen yhdisteen ympärille esimerkin mukaisesti.
# Esimerkiksi seuraavat yhdisteet.
smiles = [
  "C1CCCCC1",
  "CC(C)Cc1ccc(cc1)C(C)C(=O)O",
  "CC1(C(N2C(S1)C(C2=O)NC(=O)C(C3=CC=C(C=C3)O)N)C(=O)O)C",
  "CC1=C(C(=O)C(=C(C1=O)OC)OC)CC=C(C)CC\C=C(/C)\CC\C=C(/C)\CC\C=C(/C)\CC\C=C(/C)\CC\C=C(/C)\CC\C=C(/C)\CCC=C(C)C", 
  "F[C@H]1CC[C@H](O)CC1", 
  "C1CC1[C@H](F)C1CCC1", 
  "CC(=O)OC1=CC=CC=C1C(=O)O"] # Tämä on Aspiriini!

x = descriptors_from_smiles(smiles) # Koneoppimisen malli hyödyntää molekulaarisia deskriptoreita laskiessaan kysyttyjä arvoja SMILES:ien perusteella
display(x) # Tulostetaan taulukko.

### 11. Ennustetaan liukoisuutta

In [ ]:
# Ennustetaan käyttäen koneoppimisen mallia.
ennuste = model.predict(x)

# Luodaan taulukko ennuisteista ja niiden SMILES:sseista ja tulostetaan se.
df = pd.DataFrame({"Ennustettu LogS": ennuste, "SMILE": smiles})
display(df)

### Kysymys 10.
**Mitä yhdisteitä tutkit ja liukenevatko ne veteen?**

### Kysymys 11.
**Miten valisemasi yhdisteen liukoisuus eroaa aspiriinista, joka tunnetusti liukenee hyvin veteen ja sopii lääkkeenä suun kautta annettavaksi?**

---

### 12. SwissADME
SwissADME on ilmainen verkkotyökalu, jota käytetään arvioimaan kemiallisten yhdisteiden farmakokineettisiä ominaisuuksia. Se auttaa ennustamaan yhdisteiden imeytymistä, jakautumista, metaboliaa ja erittymistä, sekä analysoimaan niiden kemiallista reaktiivisuutta ja toksisuutta. Työkalu on erityisen hyödyllinen lääkekehityksessä, koska se tarjoaa nopean ja helpon tavan arvioida yhdisteiden potentiaalia toimia tehokkaina ja turvallisina lääkkeinä.

http://www.swissadme.ch/ 

### Kysymys 12.
**Tutki samoja yhdisteitä SwissADMEn avulla. Saatko saman ennusteen yhdisteen liukoisuudesta ja toimivuudesta lääkkeenä SwissADMENn avulla verrattuna yllä olevaan koneoppimismalliin?**

---